<a href="https://colab.research.google.com/github/jzhu1905/HeterophilicGraphLearningTutorial/blob/main/02_Part_II_Heterophily_Metrics_on_Synthetic_Graphs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Heterophily Benchmark Tutorial: Part II - Metrics on Synthetic Graphs

Sitao Luan, MILA \\
Chenqing Hua, McGill University & MILA \\
QiCheng Lu, McGill University & MILA \\
Jiaqi Zhu, DRW


##### ***LOG2024: A Tutorial on Heterophilic Graph Learning***

## Environment Setup


In [1]:
import os
import shutil

import ipywidgets as widgets
from ipywidgets import HBox

In [2]:
COLLAB_ENV = True

In [ ]:
if COLLAB_ENV:
    !git clone --recurse-submodules -j8 https://github.com/jzhu1905/HeterophilicGraphLearningTutorial.git
    !cd HeterophilicGraphLearningTutorial
    !git init
    !git submodule update --init --recursive
    shutil.copytree("HeterophilicGraphLearningTutorial/", "./", dirs_exist_ok=True)
    !rm -rf HeterophilicGraphLearningTutorial/

    !pip install torch==2.2.2
    !pip install torchaudio==2.2.2 torchdata==0.7.1 torchvision==0.17.2
    !pip install dgl -f https://data.dgl.ai/wheels/torch-2.2/cu121/repo.html
    import torch
    os.environ['TORCH'] = torch.__version__
    print(torch.__version__)
    !pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
    !pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
    !pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
    !pip install git+https://github.com/pyg-team/pytorch_geometric.git
    !pip install torch-geometric===2.1.0.post1
    !pip install networkx==2.5
    !pip install ogb
    !pip install powerlaw

# generate all GenCat synthetic graphs using Cara as base dataset
print("Generting GenCat synthetic graphs...")
%cd ./Heterophilic_Benchmarks/empirical-study-of-GNNs/
!python scripts/run_gencat_hetero_homo.py --dataset cora --n_iter 1
shutil.copytree("../GenCAT_Exp_hetero_homo/", "../../GenCAT_Exp_hetero_homo/", dirs_exist_ok=True)
!rm -rf ../GenCAT_Exp_hetero_homo/*.pt
%cd ../../


# downlaod all regular synthetic graphs
import gdown
import zipfile

url = "https://drive.google.com/file/d/1ErTr7KiNUdSrMBHpTBPQsQh7zfo5JMdX/view?usp=sharing"
gdown.download(url, "data_synthesis.zip", quiet=False, fuzzy=True)
file_path = f"./data_synthesis.zip"
with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall("./")
!rm -rf ./data_synthesis.zip
!rm -rf __MACOSX/

In [9]:
import sys
sys.path.append("./Heterophilic_Benchmarks")

from constants import HETEROPHILY_METRICS
from generate_syn_graph import generate_pa_syn_graph
from homophily_metrics_eval import eval_pa_syn_graph, eval_gencat_syn_graph, eval_regular_syn_graph
from visualize_metrics import metrics_visualization, homophily_lvl_visulization

IS_GPU_AVAILABLE = torch.cuda.is_available()

# Measure Homophily Level of Synthetic Graph


## 1. Regular Graph

[Luan et al.](https://arxiv.org/abs/2210.07606) proposed to generate regular graphs as follows:

1. $10$ graphs are generated for each of the $28$ edge homophily levels, from $0.005$ to $0.95$, with a total of $280$ graphs;
2. Every generated graph has five classes, with $400$ nodes in each class. For nodes in each class, $800$ random intra-class edges and ($\frac{800}{\text{H}_\text{edge}\;\;(\text{G})} -800$) inter-class edges are uniformly generated ;
3. The features of nodes in each class are sampled from node features in the corresponding class of the base datasets, e.g., *Cora*.

### 1.1. Evaluating the Same Graph Using Different Metrics

In [5]:
rg_h_lvl = widgets.BoundedFloatText(
    value=0.5,
    min=0.05,
    max=0.95,
    step=0.05,
    readout_format='.3f',
    description="Set Homophily Level (0.05-0.95):",
    style={'description_width': 'initial'}
)
display(rg_h_lvl)

BoundedFloatText(value=0.5, description='Set Homophily Level (0.05-0.95):', max=0.95, min=0.05, step=0.05, sty…

In [ ]:
# caculate heterophily metrics of the selected GenCat synthetic graph.
rg_metrics = eval_regular_syn_graph(rg_h_lvl.value, IS_GPU_AVAILABLE);

In [7]:
# comparing different metrics
metrics_visualization("Regular", rg_h_lvl.value,  rg_metrics)

### 1.2. Evaluating Graphs with Different Homophily Levels Using the Same Metric

In [8]:
rg_metric = widgets.Dropdown(
    options=HETEROPHILY_METRICS.values(),
    default=list(HETEROPHILY_METRICS.values())[0],
    description='Choose a homophily metric:',
    style={'description_width': 'initial'},
    layout={'width': 'max-content'}
)
display(rg_metric)

Dropdown(description='Choose a homophily metric:', layout=Layout(width='max-content'), options=('Node Homophil…

In [10]:
homophily_lvl_visulization("RG", rg_metric.value)

## 2. Preferential Attachment Syntheic Graphs

[Karimi et al.](https://www.nature.com/articles/s41598-018-29405-7) introduced homophily into the Preferential Attachment (PA) model, and [Abu-El-Haija et al.](https://arxiv.org/abs/1905.00067) extended it to multi-class settings, commonly used in graph machine learning. The graph generation process for a graph $G$ with $N$ nodes, $C$ classes, and a homophily coefficient $\mu$ is as follows:
1. Nodes are divided into $C$ equal-sized classes.
2. A new node is added iteratively, with its class $y_i$ randomly assigned.
3.  The new node connects to an existing node based on the probability $\bar{p}_{ij}$, which depends on the class labels $y_i$ and $y_j$. If $y_i = y_j$, the probability is proportional to the degree of the existing node and $\mu$. Otherwise, it is adjusted by a "cost" term $w_{d(y_i, y_j)}$ based on the class distance.
4. The connection probabilities are normalized across neighbors.
5. Node features are sampled from distinct 2D Gaussian distributions for each class.


### 2.1. Evaluating the Same Graph Using Different Metrics

In [11]:
pa_h_lvl = widgets.BoundedFloatText(
    value=0.5,
    min=0.005,
    max=0.95,
    step=0.005,
    readout_format='.3f',
    description="Set Homophily Level (0.05-0.95):",
    style={'description_width': 'initial'}
)
display(pa_h_lvl)

BoundedFloatText(value=0.5, description='Set Homophily Level (0.05-0.95):', max=0.95, min=0.005, step=0.005, s…

In [ ]:
# generate a PA synthetic graph
generate_pa_syn_graph(pa_h_lvl.value)
# caculate heterophily metrics of the generated PA synthetic graph.
pa_metrics = eval_pa_syn_graph(pa_h_lvl.value, IS_GPU_AVAILABLE);

In [13]:
# comparing different metrics
metrics_visualization("PA", pa_h_lvl.value,  pa_metrics)

### 2.2. Evaluating Graphs with Different Homophily Levels Using the Same Metric

In [14]:
pa_metric = widgets.Dropdown(
    options=HETEROPHILY_METRICS.values(),
    default=list(HETEROPHILY_METRICS.values())[0],
    description='Choose a homophily metric:',
    style={'description_width': 'initial'},
    layout={'width': 'max-content'}
)
display(pa_metric)

Dropdown(description='Choose a homophily metric:', layout=Layout(width='max-content'), options=('Node Homophil…

In [15]:
homophily_lvl_visulization("PA", pa_metric.value)

## 3. GenCat Syntheic Graphs

[GenCat](https://arxiv.org/abs/2109.04639) generates synthetic graphs based on a real-world graph and a hyperparameter $\beta$ , which controls the homophily/heterophily properties of the generated graph. Using the base graph and $\beta$ , the following are calculated:

- Class preference matrix $M^{(\beta)} \in \mathbb{R}^{C \times C}$,
- Class preference deviation matrix $D^{(\beta)} \in \mathbb{R}^{C \times C}$,
- Class size distribution,
- Attribute-class correlation matrix $H \in \mathbb{R}^{F \times C}$.

These quantities are then used to compute three latent factors:

- Node-class membership proportions $U \in [0, 1]^{N \times C}$,
- Node-class connection proportions $U' \in [0, 1]^{N \times C}$,
- Attribute-class proportions $V \in [0, 1]^{F \times C}$.

Where $C$, $F$, and $N$ represent the number of classes, features, and nodes in the base graph, respectively. The synthetic graph is then generated using these latent factors.

### 3.1. Evaluating the Same Graph Using Different Metrics

In [16]:
gc_b = widgets.BoundedFloatText(
    value=0.2,
    min=-0.1,
    max=0.8,
    step=0.1,
    readout_format='.1f',
    description="Set beta:",
    style={'description_width': 'initial'}
)
display(gc_b)

BoundedFloatText(value=0.2, description='Set beta:', max=0.8, min=-0.1, step=0.1, style=DescriptionStyle(descr…

In [ ]:
# caculate heterophily metrics of the selected GenCat synthetic graph
gencat_metrics = eval_gencat_syn_graph(gc_b.value, IS_GPU_AVAILABLE);

In [18]:
# comparing different metrics
metrics_visualization("GenCat", gc_b.value, gencat_metrics)

### 3.2. Evaluating Graphs with Different Homophily Levels Using the Same Metric

In [19]:
gencat_metric = widgets.Dropdown(
    options=HETEROPHILY_METRICS.values(),
    default=list(HETEROPHILY_METRICS.values())[0],
    description='Choose a homophily metric:',
    style={'description_width': 'initial'},
    layout={'width': 'max-content'}
)
display(gencat_metric)

Dropdown(description='Choose a homophily metric:', layout=Layout(width='max-content'), options=('Node Homophil…

In [20]:
homophily_lvl_visulization("GenCat", rg_metric.value)